In [14]:
import requests
import random
import json
import os
import tqdm
import time

In [18]:
api_key = "xxx"
def retry_with_exponential_backoff(
    func,
    initial_delay: float = 5,
    exponential_base: float = 2,
    jitter: bool = True,
    max_retries: int = 10,
):
    """Retry a function with exponential backoff."""

    def wrapper(*args, **kwargs):
        # Initialize variables
        num_retries = 0
        delay = initial_delay

        # Loop until a successful response or max_retries is hit or an exception is raised
        while True:
            response = func(*args, **kwargs)

            # Retry on specified errors
            if "error" in response:
                # Increment retries
                num_retries += 1

                # Check if max retries has been reached
                if num_retries > max_retries:
                    print("Maximum Retry Exceed")
                    return {}

                # Increment the delay
                delay *= exponential_base * (1 + jitter * random.random())

                # Sleep for the delay
                time.sleep(delay)
            else:
                return response

    return wrapper

@retry_with_exponential_backoff
def prompt_chatgpt(model_name, prompt, max_tokens, temperature, top_p=None, n=5):
    """Make an api call to ChatGPT and write the respone to a file"""
    api_url = "https://api.openai.com/v1/completions"
    headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
    data = {"model": model_name,
            "prompt": f"{prompt}",
            "temperature": temperature,
            "top_p": top_p,
            "max_tokens": max_tokens,
            "n":n}

    try:
        response = requests.post(api_url, headers=headers, json=data)
    except requests.exceptions as err:
        print(err)
        raise Exception("Error in Connection...")

    json_response = response.json()
    return json_response


In [19]:
from utils import load_json_dataset
he_data = load_json_dataset("dataset/humaneval/HumanEval.jsonl")

In [9]:
model_name = "code-davinci-002"
max_tokens = 350
temperature = 0.8
top_p = 1.0
n = 5
output_file_pth = f"dataset/humaneval/{model_name.replace('-','_')}-{temperature}-{top_p}-{max_tokens}-{n}_response.json"
# out_file = open(output_file_pth, "a")

In [7]:
# prompt = f"Complete the following function: \n{csn_data[0]['prompt']}"
# prompt = f"{csn_data[0]['prompt']}"
for data in tqdm.tqdm(he_data[:1]):
    print(data)
    prompt = data['prompt']
    response = prompt_chatgpt(model_name, prompt, max_tokens, temperature, top_p, n)
    print(response['choices'][0])
    # out_file.write(json.dumps(response) + "\n")
# out_file.close()

  0%|                                                     | 0/1 [00:00<?, ?it/s]

{'task_id': 'HumanEval/0', 'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'entry_point': 'has_close_elements', 'canonical_solution': '    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n', 'test': "\n\nMETADATA = {\n    'author': 'jt',\n    'dataset': 'test'\n}\n\n\ndef check(candidate):\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.3) == True\n    assert candidate([1.0, 2.0, 3.9, 4.0, 5.0, 2.2], 0.05) == False\n    assert candidate([1.0, 2.0, 5.9, 4.0

100%|█████████████████████████████████████████████| 1/1 [00:12<00:00, 12.45s/it]

{'text': '    for i, n in enumerate(numbers):\n        for j, m in enumerate(numbers):\n            if i == j:\n                continue\n            if abs(n - m) < threshold:\n                return True\n    return False\n\n\ndef find_closest_pair(numbers: List[float], threshold: float) -> Tuple[int, int]:\n    """ Finds the indices of the two closest numbers in given list of numbers\n        >>> find_closest_pair([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n        (2, 1)\n        >>> find_closest_pair([1.0, 2.0], 0.1)\n        (0, 1)\n    """\n    for i, n in enumerate(numbers):\n        for j, m in enumerate(numbers):\n            if i == j:\n                continue\n            if abs(n - m) < threshold:\n                return i, j\n    raise ValueError(f"No close elements found in list {numbers}")', 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}


In [13]:
a = load_json_dataset(output_file_pth)
a[5]['raw_response']

{'error': {'message': 'Rate limit reached for default-code-davinci-002 in organization org-xk6WoVlZlZk2h4fCHJMwPZGc on requests per min. Limit: 20.000000 / min. Current: 30.000000 / min. Contact support@openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.',
  'type': 'requests',
  'param': None,
  'code': None}}